In [1]:
import onnx
import tf2onnx
import numpy as np
import onnxruntime
import librosa

2023-08-20 22:11:21.602076: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-20 22:11:23.083232: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# from tensorflow.keras.models import load_model
# model_tf = load_model('model.h5')
# onnx_model, _ = tf2onnx.convert.from_keras(model_tf)
# onnx.save(onnx_model, 'model.onnx')

In [3]:
def preprocess(file_path, n_mfcc=13, n_fft=2048, hop_length=512):

    # load audio file
    signal, sr = librosa.load(file_path)
    # ensure consistency in the audio file length
    if len(signal) >= 22050:
        signal = signal[:22050]

        # extract MFCCs
        MFCCs = librosa.feature.mfcc(y=signal, n_mfcc=n_mfcc, 
                                        n_fft=n_fft, hop_length=hop_length)
        
        return MFCCs.T

In [4]:
session = onnxruntime.InferenceSession("model.onnx")

In [5]:
session.get_inputs()[0].shape

['unk__216', 44, 13, 1]

In [6]:
number = 2
TEST_AUDIO_FILE_PATH = '/home/long/Source-Code/SpechCommandAppication/archive/augmented_dataset/augmented_dataset/tree/{}.wav'.format(number)

In [7]:
input_name = session.get_inputs()[0].name


In [8]:
output_name = session.get_outputs()[0].name
output_name

'dense_1'

In [9]:
mfcc = preprocess(TEST_AUDIO_FILE_PATH)

In [10]:
test_audio_input = np.expand_dims(mfcc, axis=-1)
test_audio_input = np.expand_dims(test_audio_input, axis=0)

In [11]:
result = session.run([output_name], {input_name: test_audio_input})

In [12]:
np.argmax(result)

14